In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Open Geojson

In [2]:
germany_gdf = gpd.read_file(
    "data/georef-germany-kreis@public/georef-germany-kreis-millesime.shp"
)

In [3]:
germany_gdf["krs_code"] = germany_gdf["krs_code"].str[2:7]

## Get Open Population data

In [4]:
population_df = pd.read_csv(
    "data/12411-0015_en_flat/12411-0015_en_flat.csv",
    delimiter=";",
    dtype={"1_variable_attribute_code": str},
)

In [5]:
pop_2016_df = population_df[population_df["time"] == "2016-12-31"][
    ["1_variable_attribute_code", "value"]
].reset_index(drop=True)
pop_2024_df = population_df[population_df["time"] == "2024-12-31"][
    ["1_variable_attribute_code", "value"]
].reset_index(drop=True)

In [6]:
pop_df = pop_2016_df.merge(
    pop_2024_df, on="1_variable_attribute_code", suffixes=("_2016", "_2024")
)
pop_df = pop_df.rename(
    columns={
        "1_variable_attribute_code": "krs_code",
        "value_2016": "2016",
        "value_2024": "2024",
    }
)

In [7]:
pop_df["2016"] = pd.to_numeric(pop_df["2016"], errors="coerce")
pop_df["2024"] = pd.to_numeric(pop_df["2024"], errors="coerce")

In [8]:
pop_df["DIFF"] = pop_df["2024"] - pop_df["2016"]
pop_df["PERCENT"] = ((pop_df["DIFF"] / pop_df["2016"]) * 100.0).round(decimals=1)

## Merge data and export

In [9]:
pop_gdf = germany_gdf.merge(pop_df, on="krs_code", how="left")

In [10]:
pop_gdf = pop_gdf.to_crs(4839)
pop_gdf.to_file("data/Population_Change_Germany_2016_2024.gpkg")

In [11]:
pop_gdf["2016"].sum()

np.float64(82479065.0)

In [12]:
pop_gdf["2024"].sum()

np.float64(83577140.0)

In [13]:
pop_gdf["lan_code"] = pop_gdf["lan_code"].str[2:4]

In [14]:
pop_gdf.dissolve(by="lan_code")[["geometry"]].to_file("data/german_states.gpkg")